## Setup

Ollama is a library for running open source LLMs locally.

If you haven't already, download Ollama at (here)[https://ollama.com/]

then, open your favorite terminal and run

`ollama run llama3`

Next, lets install our python dependencies, including langchain, faiss, ollama, and firecrawl-py

In [2]:
%pip install --upgrade --quiet langchain langchain-community faiss-cpu ollama firecrawl-py

Note: you may need to restart the kernel to use updated packages.


# Load website with Firecrawl

In [ ]:
from langchain_community.document_loaders import FireCrawlLoader  # Importing the FireCrawlLoader

url = "https://firecrawl.dev"
loader = FireCrawlLoader(
    api_key="fc-30c90634837744469ef9a280b9be1703", # Note: Replace 'YOUR_API_KEY' with your actual FireCrawl API key
    url=url,  # Target URL to crawl
    mode="crawl"  # Mode set to 'crawl' to crawl all accessible subpages
)
docs = loader.load()  # This will crawl the website and return clean markdown and metadata for each accessible subpage. Returns as a langchain document

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
import ollama

# Set up Vectorstore

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = FAISS.from_documents(documents=splits, embedding=OllamaEmbeddings())

## Retrieval and Generation

In [1]:
question = "What is firecrawl?"
docs = vectorstore.similarity_search(query=question)
prompt = f"""You are a friendly assistant. You're job is to answer the users question based on the documentation provided below:
  Docs:
  
  {docs}

  Question: {question}
"""
response = ollama.chat(model='llama3:latest', messages=[
  {
    'role': 'user',
    'content': prompt,
  },
])

print(response['message']['content'])

NameError: name 'vectorstore' is not defined